In [1]:
using Oscar
using LinearAlgebra

In [2]:
function dimQI(v,lista)
    max = 0
    leadmons = Set([leading_monomial(f) for f in lista])
    for p in leadmons
        if v in vars(p)
            m = degree(p, v)
            if m> max
                max = m
            end
        end
    end
    return max
end

dimQI (generic function with 1 method)

In [3]:
function kbasis(GB, VList)
    # devuelve una lista de monomios que forman una base del anillo cociente, donde GB es una base de Gröbner para un ideal de dimensión cero, y genera un mensaje de error si el ideal no es de dimensión cero.
    
    if iszero(dim(GB))  # verifica si el ideal es de dimensión cero
        Gb = collect(gens(GB))
        B = [(div(VList[1],VList[1])) ]  # inicializa la lista de monomios de la base con el monomio 1
        leadmons = Set([leading_monomial(f) for f in Gb])  # crea un conjunto de los monomios principales de cada polinomio en la base de Gröbner
        for v in VList
            m = dimQI(v, leadmons)  # determina el grado de la variable v en el anillo cociente
            C = copy(B)  # crea una copia de la lista de monomios actual como base para la siguiente iteración
            for t in C
                for l in 1:m-1
                    t = t*v  # multiplica el monomio actual por la variable v
                    if  !(any((divrem(t,u)[2] == 0) for u in leadmons))  # verifica si alguno de los monomios principales de la base de Gröbner divide al nuevo monomio
                        push!(B, t)  # si ningún monomio principal lo divide, agrega el nuevo monomio a la lista de monomios de la base
                    end
                end
            end
        end
        return B  # devuelve la lista de monomios de la base
    else
        error("el ideal no es de dimensión cero")
    end
end

kbasis (generic function with 1 method)

In [4]:
function matrixM(variable,kbase,Gbase)
    kbx = kbase*variable
    reducida = reduce(collect(gens(kbx)),collect(Gbase))
    Mx = [coeff(kr,k) for k in collect(gens(kbase)) , kr in collect(reducida)] 
    M = [Int64.(numerator(i))//Int64.(denominator(i)) for i in Mx]
    return M
end

matrixM (generic function with 1 method)

In [5]:
function buenaCombinacion(ListaM,E0,F0)
    n, m = size(ListaM[1])
    Ei = E0
    Fi = F0
    S0 = [diag(Fi*Mi*Ei) for Mi in ListaM]
    S = hvcat(length(ListaM), S0...)
    sigma = [ℯ^(1*im*(t-1)*π/n) for t in 1:n]
    Σ=diagm(sigma)
    alpha=pinv(S)*sigma
    M =sum([alpha[i]*ListaM[i] for i in 1:length(ListaM)])
    return M,Σ
end

buenaCombinacion (generic function with 1 method)

In [6]:
function XYS(Σ,Z,Δ)
    n, m = size(Z)
    S = diagm(diag(Δ-Z*Σ))
    X = [i != j ? (-Δ[i,j]+ Z[i,j] * Σ[j,j])/(Σ[i,i]-Σ[j,j]) : 0 for i in 1:n , j in 1:n]
    Y = [i != j ? (Δ[i,j]- Z[i,j] * Σ[i,i])/(Σ[i,i]-Σ[j,j]) : -Z[i,i] for i in 1:n , j in 1:n]
    return X,Y,S
end

XYS (generic function with 1 method)

In [7]:
function iteracionNewton(E,F,Σ,M)
    n, m = size(M)
    I_n = Matrix{Float64}(I, n, n)
    Z = F*E-I_n
    Δ = F * M * E - Σ
    X,Y,S = XYS(Σ,Z,Δ)
    Em1 = E * (I_n + X)
    Fm1 = (I_n + Y) * F
    Σm1 = Σ + S
    return Em1,Fm1,Σm1
end

iteracionNewton (generic function with 1 method)

In [8]:
function newtonDiagSimul(M,E0,F0,Σ0)
    n, m = size(M)
    Ei = E0
    Fi = F0
    Σi = Σ0
    err = opnorm((Fi*M*Ei-Σi), Inf)
    errores = [err]
    numbern = 0
    while err > 10^(-15) 
        Ei,Fi,Σi = iteracionNewton(Ei,Fi,Σi,M)
        err = opnorm((Fi*M*Ei-Σi), Inf)
        numbern+=1
        append!(errores, err)
    end
    return Ei,Fi,numbern,Σi, errores
end

newtonDiagSimul (generic function with 1 method)

In [9]:
R, x = PolynomialRing(QQ, ["x$i" for i in 1:3])

(Multivariate Polynomial Ring in x1, x2, x3 over Rational Field, QQMPolyRingElem[x1, x2, x3])

In [10]:
f1 = x[1]^2 - 2 * x[1] * x[3] + 5
f2 = x[1] * x[2]^2 + x[2] * x[3] + 1
f3 = 3 * x[2]^2 - 8 * x[1] * x[3]
Id = ideal(R,[f1,f2,f3])

ideal(x1^2 - 2*x1*x3 + 5, x1*x2^2 + x2*x3 + 1, -8*x1*x3 + 3*x2^2)

In [11]:
G = groebner_basis(Id)

Gröbner basis with elements
1 -> 3*x2^2 - 8*x1*x3
2 -> x1^2 - 2*x1*x3 + 5
3 -> 160*x3^3 - 160*x1*x3 + 415*x2*x3 + 12*x1 - 30*x2 - 224*x3 + 15
4 -> 240*x2*x3^2 - 9*x1*x2 + 1600*x1*x3 + 18*x2*x3 + 120*x3^2 - 120*x1 + 240*x3
5 -> 16*x1*x3^2 + 3*x2*x3 - 40*x3 + 3
6 -> 3*x1*x2*x3 - 6*x2*x3^2 - 40*x1*x3 + 3*x1 - 6*x3
with respect to the ordering
degrevlex([x1, x2, x3])

In [12]:
Gi = ideal(G) 

ideal(-8*x1*x3 + 3*x2^2, x1^2 - 2*x1*x3 + 5, -160*x1*x3 + 12*x1 + 415*x2*x3 - 30*x2 + 160*x3^3 - 224*x3 + 15, -9*x1*x2 + 1600*x1*x3 - 120*x1 + 240*x2*x3^2 + 18*x2*x3 + 120*x3^2 + 240*x3, 16*x1*x3^2 + 3*x2*x3 - 40*x3 + 3, 3*x1*x2*x3 - 40*x1*x3 + 3*x1 - 6*x2*x3^2 - 6*x3)

In [13]:
KB = kbasis(Gi,gens(R))

8-element Vector{QQMPolyRingElem}:
 1
 x1
 x2
 x1*x2
 x3
 x3^2
 x1*x3
 x2*x3

In [14]:
kb = ideal(R,KB)

ideal(1, x1, x2, x1*x2, x3, x3^2, x1*x3, x2*x3)

In [15]:
ListaM = [matrixM(xi,kb,G) for xi in gens(R) ]

3-element Vector{Matrix{Rational{Int64}}}:
 [0//1 -5//1 … -3//8 0//1; 1//1 0//1 … 0//1 0//1; … ; 0//1 2//1 … 0//1 0//1; 0//1 0//1 … -3//8 -3//20]
 [0//1 0//1 … 0//1 -1//2; 0//1 0//1 … 0//1 0//1; … ; 0//1 0//1 … 0//1 0//1; 0//1 0//1 … -3//20 -1//2]
 [0//1 0//1 … -3//16 0//1; 0//1 0//1 … 0//1 1//2; … ; 0//1 1//1 … 0//1 -20//3; 0//1 0//1 … -3//16 -3//40]

In [16]:
ListaM[1]

8×8 Matrix{Rational{Int64}}:
 0//1  -5//1  0//1   0//1   0//1  -3//16  -3//8   0//1
 1//1   0//1  0//1   0//1   0//1   0//1    0//1   0//1
 0//1   0//1  0//1  -5//1   0//1   0//1    0//1   0//1
 0//1   0//1  1//1   3//20  0//1   0//1    0//1   3//40
 0//1   0//1  0//1   0//1   0//1   5//2    0//1   0//1
 0//1   0//1  0//1  -2//1   0//1   0//1    0//1  -1//1
 0//1   2//1  0//1   0//1   1//1   0//1    0//1   0//1
 0//1   0//1  0//1  -3//10  0//1  -3//16  -3//8  -3//20

In [17]:
L0,E0 = eigen(ListaM[1])
F0 = inv(E0)
M0,Σ0 = buenaCombinacion(ListaM,E0,F0)

(ComplexF64[0.0 + 0.0im 0.3545782372370618 - 0.4758080016548146im … 0.07201654684276687 - 0.0394194655777657im -0.04023509791423063 - 0.022965533214129547im; -0.07091564744741236 + 0.09516160033096292im 0.0 + 0.0im … 0.0 + 0.0im -0.12112847746663263 + 0.00995697454307895im; … ; 0.0 + 0.0im -0.38408824982809 + 0.21023714974808375im … 0.0 + 0.0im 1.6150463662217684 - 0.132759660574386im; 0.0 + 0.0im 0.0 + 0.0im … 0.05994601746849768 - 0.04630912554200456im -0.011428479177123887 - 0.03873331944523583im], ComplexF64[1.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im 0.0 + 0.0im; 0.0 + 0.0im 0.9238795325112867 + 0.3826834323650898im … 0.0 + 0.0im 0.0 + 0.0im; … ; 0.0 + 0.0im 0.0 + 0.0im … -0.7071067811865475 + 0.7071067811865476im 0.0 + 0.0im; 0.0 + 0.0im 0.0 + 0.0im … 0.0 + 0.0im -0.9238795325112867 + 0.3826834323650899im])

In [18]:
E,F,niter,Sig,errores = newtonDiagSimul(M0,E0,F0,Σ0)

(Any[0.05272743026516138 - 9.886987634482231e-18im 0.005766300683815994 + 0.04752411334956926im … 0.30180956937978076 + 0.32523385345788214im -0.0458809740266818 - 1.684905997451156e-18im; -0.04789102505995172 - 1.3935979435344397e-17im -0.05119229538436638 + 0.0017120312663657963im … 0.14941466924701835 - 0.12944321585391327im -0.04750997434806928 - 1.2137076579139142e-18im; … ; 0.6555225171630593 + 1.191988577014204e-16im 0.6861866286348454 + 7.747211250833134e-17im … -0.24169685203515937 - 0.160450291976819im 0.6179989175052414 - 5.3618621129116944e-18im; 0.3115677649436785 + 1.871055540796989e-17im -0.015585585517088323 - 0.2255493601243789im … 0.06495531335086377 - 0.01290805781621749im -0.2517785503653274 - 8.318451025021215e-18im], ComplexF64[0.11532509941886536 - 7.31359529851153e-17im -0.1269715177284023 + 1.071090885066642e-16im … 0.3582097891545124 - 2.151953192881275e-17im 0.9363671068932042 + 1.428817077232005e-16im; 0.18863275855630882 - 0.03067326534653552im -0.043932941

In [20]:
puntos = [diag(F*Mi*E) for Mi in ListaM]
puntosCompilados = hvcat(length(ListaM), puntos...)

8×3 Matrix{Any}:
   -1.10099+1.84889e-32im     -2.878+2.46519e-32im   -2.82118+0.0im
 -0.0815021-0.931071im       2.34979+0.0430587im    -0.274005+2.19913im
 -0.0815021+0.931071im       2.34979-0.0430587im    -0.274005-2.19913im
  0.0724908-2.237im        -0.465772+0.464209im     0.0724227-0.00210322im
  0.0724908+2.237im        -0.465772-0.464209im     0.0724227+0.00210322im
  0.0766489-2.24312im        0.46123-0.497027im     0.0763638-0.0083441im
  0.0766489+2.24312im        0.46123+0.497027im     0.0763638+0.0083441im
   0.965712+5.54668e-32im    -2.8125-1.57156e-31im    3.07162+1.75645e-31im